# 12.MNIST 손글씨 데이터 분류 (CNN 버전)
### 미 국립표준기술원(NIST)이 고등학생과 인구조사국 직원 등이 직접 쓴 손글씨 데이터
* 70,000 건의 0~9까지 흑백 손글씨 숫자 이미지 데이터
    - 60,000 건은 훈련 데이터로
    - 10,000 건은 테스트 데이터로 지정되어 있음

In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


# 데이터 불러오기

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

### 이미지 데이터 세트를 CNN에서 처리하려면 4차원 데이터이어야 함

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [ ]:
X_train.shape

### 컨볼루션 신경망의 설정

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### 모델 학습 자동 중단 설정

In [ ]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

### 모델 학습 수행

In [ ]:
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=30, batch_size=200, verbose=0, callbacks=[early_stopping_callback, checkpointer])

### 테스트 데이터 셋으로 모델의 성능 평가

In [ ]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

### 결과 검토

In [ ]:
# 검증 셋의 오차와 정확도
y_vloss = history.history['val_loss']
y_vacc = history.history['val_acc']

# 학습셋의 오차와 정확도
y_loss = history.history['loss']
y_acc = history.history['acc']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Validation_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Training_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vacc, marker='.', c="red", label='Validation_acc')
plt.plot(x_len, y_acc, marker='.', c="blue", label='Training_acc')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')

### 모델 저장

In [ ]:
model.save('./mnist_model.h5')